In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import cv2
import numpy as np
import time

# Import MediaPipe drawing utilities
from mediapipe.framework.formats import landmark_pb2


# model_path = '/Users/daniel/Documents/Python-Projects/mediapipe-test/hand_landmarker.task'
model_path = 'hand_landmarker.task'
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode


def letterbox_to_square(img, color=(0, 0, 0)):
    h, w = img.shape[:2]
    if h == w:
        return img
    size = max(h, w)
    top = (size - h) // 2
    bottom = size - h - top
    left = (size - w) // 2
    right = size - w - left
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)


def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        
        # Calculate joint angles
        angles = get_finger_angles(hand_landmarks)
        
        # Print angles to console
        print(f"Hand {idx} angles:")
        for joint, angle in angles.items():
            print(f"  {joint}: {angle:.1f}°")

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        
        mp.solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            mp.solutions.hands.HAND_CONNECTIONS,
            mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
            mp.solutions.drawing_styles.get_default_hand_connections_style())

    return annotated_image

# Global variable to store the latest annotated frame
latest_annotated_frame = None

# Create a hand landmarker instance with the live stream mode:
def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    global latest_annotated_frame
    if result.hand_landmarks:
        latest_annotated_frame = draw_landmarks_on_image(output_image.numpy_view(), result)
    else:
        latest_annotated_frame = output_image.numpy_view()

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)

cap = cv2.VideoCapture(0)

with HandLandmarker.create_from_options(options) as landmarker:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Letterbox to square to avoid non-square ROI warnings
        square_frame = letterbox_to_square(frame)

        # Convert BGR to RGB (OpenCV uses BGR, MediaPipe uses RGB)
        rgb_frame = cv2.cvtColor(square_frame, cv2.COLOR_BGR2RGB)
        
        # Convert the frame received from OpenCV to a MediaPipe's Image object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        
        # Get current timestamp in milliseconds
        timestamp_ms = int(time.time() * 1000)
        
        # Process the frame
        landmarker.detect_async(mp_image, timestamp_ms)

        # Display the annotated frame if available
        if latest_annotated_frame is not None:
            # Convert RGB back to BGR for OpenCV display
            display_frame = cv2.cvtColor(latest_annotated_frame, cv2.COLOR_RGB2BGR)
            cv2.imshow('Hand Tracking', display_frame)
        else:
            cv2.imshow('Hand Tracking', square_frame)
        
        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1768339483.998741 8712501 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1768339484.046269 8712785 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768339484.078114 8712793 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768339484.130519 8712790 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


: 

In [ ]:
def calculate_angle(a, b, c):
    """Calculate angle between three points"""
    a = np.array(a)  # First point
    b = np.array(b)  # Vertex point
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

def get_finger_angles(hand_landmarks):
    """Extract finger joint angles from hand landmarks"""
    landmarks = [[lm.x, lm.y] for lm in hand_landmarks]
    
    angles = {}
    
    # Index finger angles
    angles['index_mcp'] = calculate_angle(landmarks[0], landmarks[5], landmarks[6])
    angles['index_pip'] = calculate_angle(landmarks[5], landmarks[6], landmarks[7])
    angles['index_dip'] = calculate_angle(landmarks[6], landmarks[7], landmarks[8])
    
    # Middle finger angles
    angles['middle_mcp'] = calculate_angle(landmarks[0], landmarks[9], landmarks[10])
    angles['middle_pip'] = calculate_angle(landmarks[9], landmarks[10], landmarks[11])
    angles['middle_dip'] = calculate_angle(landmarks[10], landmarks[11], landmarks[12])
    
    # Ring finger angles
    angles['ring_mcp'] = calculate_angle(landmarks[0], landmarks[13], landmarks[14])
    angles['ring_pip'] = calculate_angle(landmarks[13], landmarks[14], landmarks[15])
    angles['ring_dip'] = calculate_angle(landmarks[14], landmarks[15], landmarks[16])
    
    # Pinky finger angles
    angles['pinky_mcp'] = calculate_angle(landmarks[0], landmarks[17], landmarks[18])
    angles['pinky_pip'] = calculate_angle(landmarks[17], landmarks[18], landmarks[19])
    angles['pinky_dip'] = calculate_angle(landmarks[18], landmarks[19], landmarks[20])
    
    return angles